# Images and Histograms 

## Learning Outcomes

By the end of this notebook you will know how to:

- Treat images as arrays
- Manipulate images.
- Plot images. 
- Compute histograms and plot them.

## Treating Images As Arrays

In the simplest terms an image is a 2D array of values representing intensity in each pixel. Hopefully the parallel to a 2D `numpy` array here should immediately come to mind. This means we can manipulate, probe and produce images using many of the processes we have already covered. 

### Producing images

To work with images we first need a way to see them. This is where `matplotlib` comes to the rescue. In addition to the `plot` and `scatter` functions you have already seen, there is also a function for plotting images `imshow`. To use it you simply supply your image as an argument. Lets set up a random 2D array and plot it as an image.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Create an iamge of 1000**2 random values
img = np.random.uniform(1, 100, (1000, 1000))

# Plot the image
plt.imshow(img)
plt.show()



Notice how `matplotlib` has attributed axis values to the image despite us giving it no information but the x and y ranges. What `matplotlib` has done is plot the number of pixels along each axis. Let's say we know that the extent of the x and y axes is 200 with the pixels centred on 0 in both axes. We can tell `matplotlib` this is the case using the `extent` keyword argument to `imshow`. 

In [ ]:
# Plot the image with an extent
plt.imshow(img, extent=[-100, 100, -100, 100])

# Label the axes
plt.xlabel(r'$x$ (arbitrary units)')
plt.ylabel(r'$y$ (arbitrary units)')

plt.show()

The list passed to the `extent` keyword argument is a list of the extremes along each axis of the form `[xmin, xmax, ymin, ymax]`.

Quite often you actually just want to see the image with none of that extra gubbins. In that case you can simply turn off the axes, although there are many ways to do this the simplest is to do the following. 

In [ ]:
# Plot the image
plt.imshow(img)

# Remove the axes
plt.axis(False)

plt.show()

The final thing to note is how to apply colormaps. These allow you to make your grayscale images (single values in pixels, rather than RGB arrays with 3 values in each pixel) "look prettier". A full list of the available colormaps can be found [here](https://matplotlib.org/tutorials/colors/colormaps.html). To use a colormap you simply supply the name of that colormap to the `cmap` keyword argument, as shown below.

In [ ]:
# Plot the image with the plasma colormap
plt.imshow(img, cmap='plasma')

# Remove the axes
plt.axis(False)

plt.show()


You can use this simple set of functions and arguments to display any images you want. When combined with a way to save plots (`plt.savefig`) you can later combine images and make movies. This is exactly what was used to make some of the [movies](https://www.youtube.com/shorts/20yUUrs_TUI) I've made from cosmological simulations.

### Manipulating images

We can treat these images as simple arrays and perform any operation on them that you could on an array. While this provides a wealth of operations in its own right there's plenty more! Beyond this, there are a large number of image processing specific packages such as `PIL`, `openCV` and `scikit-image`. We won't go into those here but there's a plethora of image processing possibilities. Here we will just demonstrate some simple operations.

Lets make a more "interesting" image to do this, a 2D Gaussian (interesting is a relative term). These can be used as an approximation for a poorly resolved point source, or star. To do this we can define a 2D Gaussian function and x and y ranges. From these we can calculate the image with a "dirty trick" to avoid looping.


In [ ]:
def gauss2d(x, y, mx=0, my=0, sigx=1, sigy=1):
    """
    A function that computes a 2D gaussian for an 
    x and y input.

    Args:
        x (float)
            The x value to calculate at.
        y (float)
            The y value to calculate at.
        mx (float)
            The mean along the x direction.
        my (float)
            The mean along the y direction.
        sigx (float)
            The standard deviation along the x direction.
        sigy (float)
            The standard deviation along the y direction.
    """

    # Compute the normalisation term
    norm = 1. / (2. * np.pi * sigx * sigy)

    # Compute the exponent
    exponent = -((x - mx)**2. / (2. * sigx**2.) 
                    + (y - my)**2. / (2. * sigy**2.))
    
    return norm * np.exp(exponent)
    
# Define the x and y values
xs = np.linspace(-5, 5, 1000)
ys = np.linspace(-5, 5, 1000)

# Combine the x and y ranges into grids (this is the dirty trick)
# This gives us the x-y coordinates of ever pixel by "meshing"
# together the input xs and ys. Feel free to print it to see
# What it produces.
xx, yy = np.meshgrid(xs, ys)

# Compute the gaussian image leaving the default values of 0 for the 
# means and 1 for the standard deviations
img = gauss2d(xx, yy)

# Plot the image with the known extent and inferno colormap
plt.imshow(img, extent=[-5, 5, -5, 5], cmap='inferno')

# Label axes
plt.xlabel(r'$x$ (arbitrary units)')
plt.ylabel(r'$y$ (arbitrary units)')

plt.show()


We can do simple calculations (sum, standard deviation of pixel values, the mean/median pixel value etc.) on the image but you can play around with that yourself. A more immediately useful function to apply is the base 10 logarithm, this can be very useful for scaling the image and bringing out detail in pictures where a bright set of pixels dominates the image.

In [ ]:
# Scale the image logarithmically
logimg = np.log10(img)

# Plot the image with the known extent and inferno colormap
plt.imshow(logimg, extent=[-5, 5, -5, 5], cmap='inferno')

# Label axes
plt.xlabel(r'$x$ (arbitrary units)')
plt.ylabel(r'$y$ (arbitrary units)')

plt.show()

We can also apply Boolean conditions to control specific regions in the image. Lets remove the background, to do this we can simply define an intensity threshold below which we set the pixel values to `np.nan`, `matplotlib` will then interpret these values as transparent.

In [ ]:
# Set any values below 10% of the maximum of the image to be 
# transparent
trans_img = img.copy()
trans_img[img < img.max() * 0.1] = np.nan

# Plot the image with the known extent and inferno colormap
plt.imshow(trans_img, extent=[-5, 5, -5, 5], cmap='inferno')

# Label axes
plt.xlabel(r'$x$ (arbitrary units)')
plt.ylabel(r'$y$ (arbitrary units)')

plt.show()



One final operation to consider, although it's not of much use here, is the clipping of an image between minimum and maximum intensity values. This will set any values below the minimum to the minimum and any values above the maximum to the maximum. This can be done a number of ways but the simplest is applying a minimum and maximum with `vmin` and `vmax`.

In [ ]:
# Plot the image with the known extent and inferno colormap, 
# setting minimum and maximum intensities
plt.imshow(img, extent=[-5, 5, -5, 5], cmap='inferno', 
                vmin=img.max()*0.5, vmax=img.max()*0.8)

# Label axes
plt.xlabel(r'$x$ (arbitrary units)')
plt.ylabel(r'$y$ (arbitrary units)')

plt.show()


### Applying a colorbar

These images are all well and good but we currently don't have a way of portraying the pixel values themselves. This can be done by including a colorbar. To do so we simply take the output of `imshow` and pass it to the colorbar function.

In [ ]:
# Plot the image with the known extent and inferno colormap, 
# setting minimum and maximum intensities
im = plt.imshow(img, extent=[-5, 5, -5, 5], cmap='inferno', 
                vmin=img.max()*0.5, vmax=img.max()*0.8)

# Label axes
plt.xlabel(r'$x$ (arbitrary units)')
plt.ylabel(r'$y$ (arbitrary units)')

plt.colorbar(im)

plt.show()

If we want we can also label the colorbar which is easily done by assigning the colorbar to a variable and setting its label. This is actually a very small taste of an object orientated approach to matplotlib which I've ignored in this course, feel free to ask for more information.

In [ ]:
# Plot the image with the known extent and inferno colormap, 
# setting minimum and maximum intensities
im = plt.imshow(img, extent=[-5, 5, -5, 5], cmap='inferno', 
                vmin=img.max()*0.5, vmax=img.max()*0.8)

# Label axes
plt.xlabel(r'$x$ (arbitrary units)')
plt.ylabel(r'$y$ (arbitrary units)')

cbar = plt.colorbar(im)
cbar.set_label("Pixel values")

plt.show()

### Exercises 13.1
We can now apply this to some exercises with a more interesting image.

- Locate the file `"Hubbleimg.pck"` in the data directory. 
- Open it (as shown in [notebook 10](10_data.ipynb) and assign the contents to a variable. 
- Plot the array you just stored in the variable as shown above. You shouldn't see anything in the image as this is a raw image with minimal processing. This is an image of the pillars of creation taken using the Hubble Space Telescope, although you wouldn't know that at this point.
- Try scaling the image with different mathematical functions to bring out detail and plot your tests. (Hint: log10 scaling is a good start but there are many many possibilities, have a play, I personally like arcsinh).
- Use `vmin` and `vmax` to improve the quality of your image and bring out the maximum detail possible.
- (No solution) Making these images look good is more art than science. Try to make the best image possible by applying anything you can think of to help. (Hint: You might be able to reduce noise by subtracting the mean or median and re-normalising the values, or maybe there's a different scaling you could apply, maybe even look into `PIL` at the image processing possible with it).




## Histograms


In previous exercises I've asked you to create histograms as with a bit of brute force it's easy enough to code one up using dictionaries or lists. This said, they are a very powerful analysis tool and come up again and again when analysing data, I'm sure you have come across them in the labs. To make a histogram you need to sort data into a set of bins. In Python there are a number of ways to produce histograms, here I will show the `numpy` method, there is nothing wrong with any other method and the `matplotlib` method is similar and produces a plot at the same time, but I feel the `numpy` method provides a bit more flexibility before plotting.

To produce a histogram we first need some data to histogram, for this we can just produce random values from a normal distribution, and then we need bins in which to sort it. To begin with we can just tell `numpy` that we want a certain number of bins, a good rule of thumb if you don't explicitly know how many bins you need is to start with the square root of the number of data points you have and go from there.

In [ ]:
# Define an array of values to histogram
vals = np.random.normal(0, 10, 10000)

# Compute the number of bins (this must be an integer)
nbin = int(np.sqrt(vals.size))


Now that we have the data and bins we can call `numpy`s histogram function which will return the bin counts and bin edges.

In [ ]:
# Compute the histogram 
H, bin_edges = np.histogram(vals, bins=nbin)


We now have the values sorted into bins where `H` is the number of counts in each bin and `bin_edges` is, unsurprisingly, the edges of the bins. These variable names are convention, you can obviously change these. Lets plot these results as a bar graph using another new matplotlib function, `bar`.



In [ ]:
# Plot the histogram as a bar graph
plt.bar(bin_edges, H)
plt.show()




Of course, you may have seen this coming... but we have an error, specifically `ValueError: shape mismatch: objects cannot be broadcast to a single shape`. This is because `H` and `bin_edges` are different sizes. `bin_edges` contains the edges of the bins, i.e. 1 more entry than `H`. We want to plot against the centre of each bin. Not to worry though, this is easily achieved.


In [ ]:
# Compute the bin centres
bin_cents = (bin_edges[:-1] + bin_edges[1:]) / 2

# Plot the histogram as a bar graph
plt.bar(bin_cents, H)

# Label axes
plt.xlabel(r'$x$')
plt.ylabel(r'$N$')

plt.show()


We know have a histogram represented as a bar plot. Notice how simply by histograming the values we have roughly reproduced the normal distribution the values were pulled from.

Another way to achieve the same result is by defining the bins ahead of time, this gives the histogram an explict range.

In [ ]:
# Define an array of values to histogram
vals = np.random.normal(0, 10, 10000)

# Compute the bins
bins = np.linspace(-40, 40, 50)

# Compute the histogram 
H, bin_edges = np.histogram(vals, bins=bins)

# Compute the bin centres
bin_cents = (bin_edges[:-1] + bin_edges[1:]) / 2

# Plot the histogram as a bar graph
# See here I'm using some keyword arguments to make the plot look nicer
plt.bar(bin_cents, H, width=bins[1:] - bins[:-1], alpha=0.8, edgecolor="grey")

# Label axes
plt.xlabel(r'$x$')
plt.ylabel(r'$N$')

plt.show()


This gives roughly the same result but now we have set the bins explictly and I've applied some simple formating to improve the plot.

### 2D Histograms

We can also produce 2D histograms which are themselves just images, think of a camera collecting photons (counts) in pixels (bins). We can approximate the Gaussian distribution above very simply by creating a 2D histogram in the same way we just produced a 1-dimensional histogram. Here we will set the number of bins as we did originally (this will apply along both axes) and also define a range for the bins to cover, this is similar to providing an array of bins but instead defines the range in which you put `nbin` bins, in a similar manner to defining the extent of an image in `imshow`.



In [ ]:
# Define the x and y values from a normal distribution
xs = np.random.normal(0, 1, 100000)
ys = np.random.normal(0, 1, 100000)

# Define the number of bins
nbin = 100

# Define the range for the bins
binrange = ((-5, 5), (-5, 5))

# Compute the histogram using the range
H, xbin_edges, ybin_edges = np.histogram2d(xs, ys, bins=nbin, 
                                                range=binrange)

# Plot the image
plt.imshow(H, cmap='viridis', extent=[-5, 5, -5, 5])

# Label axes
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')

plt.show()



This can be very useful for creating images from 1D spatial data. 

### Exercises 13.2

1. Find and define a function that will sample a log normal distribution and generate an array of 10000 random values. Now histogram this dsitribution and produce a plot.
2. Using the Gaussian histogram above can you find the standard deviation of the pixel values? 
3. Take the gaussian histogram from above, create and array the same size as `H` and populate it with random values sampled from a normal distribution centred on 0 and with a standard deviation equal to 0.5 times the answer to Q2. Add this to the image. This what you have now made is a "fake galaxy observation" with a random noise field.